### KEM
This is a simulation of an economic micro interations, in which each time step two actors, which are chosen by an interaction function, engage in a transaction that exchanges wealth between them according to a specific transaction function.

This code has been modified from https://nbviewer.jupyter.org/url/norvig.com/ipython/Economics.ipynb

In [1]:
import random
%matplotlib inline
import matplotlib.pyplot as plt
import statistics
import warnings
import numpy as np
import copy
import scipy.stats
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
warnings.filterwarnings("ignore")

In [1]:
# For population size analysis

class General_Model_MultiCity(object):

    def run_many_pops(self,obj,mu=50.):
        """
        Inputs:
            obj. Super_General_Model. The class of the model type you want to analyze. 
                Example: General_Model_MultiCity.run_many_pops(KEM_Grid) (note that you should not instantiate the KEM_Grid class)
            mu. float. the average wealth of agents
        
        Output: None
        
        Runs model simulations of varying population sizes. 
        Currently distributes city population sizes according to a zipf's law distribution
    
        """
        # generated by zipf's law distribution
        self.hists = []
        self.popsizes = np.unique(np.random.zipf(1.4,300)*10)[5:-14:3]
        #self.popsizes = [60,90,120,150,180,210,280,340,390,460,520,580,730,840,910,1160,1330,1610, 1820, 2910, 3970, 5130, 7430]
        print("popsizes:",self.popsizes)
        sims = {}
        for popsize in self.popsizes:
            # instantiate model class
            sims[popsize] = obj(N=popsize,mu=mu)
            print("\n NEW SIM\npopulation = ",popsize)
            #TODO: convergence function
            results = sims[popsize].run(k=850,convergence="auto")
            # append distribution from last time step

        
    def animate_population_dists(self):
        """
        
        Input: None
        
        Output: animation object
        
        Create animations of distributions. Returns animation function. 
        Run "HTML(anim.to_jshtml())" on return anim to display animation
        """
        
        fig = plt.figure()
        def animate(i):
            #label = label + ': G=' + str(round(gini(population), 2))
            plt.cla()
            lbl = "population = "+str(self.popsizes[i])
            plt.hist(self.hists[i], alpha = 0.4, bins = 300, label=lbl)
            plt.legend()
            plt.xlabel('Wealth')
            plt.ylabel('Count')
            plt.grid(True)

        anim = FuncAnimation(fig, animate, #init_func=init,
                                       #frames=76)#), interval=20)#, blit=True)
                                       frames=len(self.hists))#), interval=20)#, blit=True)
#         HTML(anim.to_jshtml())
        return anim

In [2]:
# General Simulation Class

class Super_General_Model(object):
    
    def __init__(self,N=500,mu=50.):
        self.debug = False
        self.N = N
        self.populate(N,mu)
        self.time = 0
        self.mu = mu
    
    def populate(self):
        """
        initialize population with distribution of income and parameters (e.g. lambda) applied to agents
        within structure (e.g. grid)
        
        """
        pass
    
    
    class agent:
        def __init__(self,income=0.):
            self.income = income      
        
    
    def choose_agents(self):
        """
        Input: None
        
        Output: A pair of agents who are going to interact. This becomes the input to interact
        """
        pass
    
    def interact(self,agents):
        """
        Input: Return value of choose_agents, the agents to interact
        
        Output: list of agent incomes
        
        Changes selected agent incomes/wealths according to KEM interaction mechanism and return new distribution
        """
        pass

    
    def get_income_dist(self):
        """
        returns the income distribution as a list of incomes
        """
        pass
   

    def simulate(self,debug=False):
        '''
        Simulate takes a certain initial population, and makes them interact.

        Inputs:
            population. list. Initial income distribution
            # steps. int. Time steps to execute the simulation
            transaction. function. function that describes the dynamic of the transaction.
            interaction. function. function that describes the dynamic of the interaction.

        Outputs:
            population after steps number of steps. 

        '''

        while(true):
            to_interact = self.choose_agents()
            new_distribution = self.interact(to_interact)
            
            ### Debugging code
            if self.debug:
                prev = np.array(self.get_income_dist())
                print("\n\n\nprev:",prev)
                print("sum:",sum(self.get_income_dist()))
                #diff = (np.array(new_distribution) - prev)/prev
                diff = (np.array(new_distribution) - prev)
                nz = np.nonzero(diff)
                lams = np.array([a.lambda_ for a in self.population])
                for dif,lam in zip(diff[nz],lams[nz]):
                    if dif < 0:
                        if dif > (1-lam):
                            print("I THINK THAT'S WRONG")
                print("diff",diff)
                population = self.get_income_dist()
                print("new:",population)
                print("sum:",sum(population))
                
            yield new_distribution

    def simulate(self,steps=100):
        '''
        Simulate takes a certain initial population, and makes them interact.

        Inputs:
            population. list. Initial income distribution
            # steps. int. Time steps to execute the simulation
            transaction. function. function that describes the dynamic of the transaction.
            interaction. function. function that describes the dynamic of the interaction.

        Outputs:
            population after steps number of steps. 

        '''

#         for n in range(steps):
        #print("in simulate, n = ",n)
        to_interact = self.choose_agents()
        new_distribution = self.interact(to_interact)

        ### Debugging code
        if self.debug:
            prev = np.array(self.get_income_dist())
            print("\n\n\nprev:",prev)
            print("sum:",sum(self.get_income_dist()))
            #diff = (np.array(new_distribution) - prev)/prev
            diff = (np.array(new_distribution) - prev)
            nz = np.nonzero(diff)
            lams = np.array([a.lambda_ for a in self.population])
            for dif,lam in zip(diff[nz],lams[nz]):
                if dif < 0:
                    if dif > (1-lam):
                        print("I THINK THAT'S WRONG")
            print("diff",diff)
            population = self.get_income_dist()
            print("new:",population)
            print("sum:",sum(population))

        return new_distribution

                
          
    def reset(self):
        self.populate(self.N,self.mu)
        
        
                
    def calc_dmdt(self,prev_dist,new_dist):
        """
        returns a 3-element np array of absolute value difference in 2nd, 3rd, and 4th moments between two distributions 
        """
        mdifs = []
        for n in [2,3,4]:
            mold = scipy.stats.moment(prev_dist,moment=n)
            mnew = scipy.stats.moment(new_dist,moment=n)
            mdif = np.abs(mold - mnew)/mnew
            mdifs.append(mdif)
        return np.array(mdifs)
            
            
        
    def run(self, k=500,convergence="manual",auto_converge_param=0.01,**kwargs):

        '''
        Run a simulation for k*N steps, returning results
        
        Inputs:
            k. int. number of transactions per person to run
            convergence. str. how to know when simulation has converged. Options are:
                - "auto": measure convergence and stop once dm/dt < converge_param for all moments m up to the 4th
                - "manual": stop after k*N steps; don't look at convergence metric
            auto_converge_param. float. Used in auto convergence to determine when distribution has converged

        '''
        N = self.N
        start = self.get_income_dist()

        self.results = [start]
        #TODO: Can this be optimized?
        
        t = 0
        #max_k = 1000000
        t_limit = 7000.*N/2.
        converged = False
        prevdist = self.get_income_dist()
        dmdt0 = [0,0,0]
        while(True):
            newdist = self.simulate()
           
            # at time t, number of interactions per person. More meaningful metric of time elapsed
            n_ints_pp = int(t / (N/2))
            # TODO: test this auto convergence
            if convergence == "auto":
                # only do stuff when n_ints_pp increases by one
                if t % (N/2) == 0:
                    # print("at t:",t)
                    # after ~1 interaction per agent
                    # calculate initial change in moments
                    if n_ints_pp == 1:
                        dmdt0 = self.calc_dmdt(prevdist,newdist)
                    elif n_ints_pp > 1:
                        dmdt = self.calc_dmdt(prevdist,newdist)
                        # if changes in all of 2nd, 3rd, and 4th moments fall below threshold, end loop
                        if t > t_limit:
                            print("WARNING: Could not converge before %s timesteps!"%t)
                            break
                        if all(dmdt/dmdt0 <= auto_converge_param):
                            print("Converged! at time ",t)
                            convergence_n = t
                            break
                        prevdist = newdist

            elif convergence == "manual":
                # stop at 
                if t >= k*N:
                    break

            # TODO: Do we want to use this? Maybe an option argument of whether or not to use it?
            # append results every 
#             if t % (N / 10) == 0:
            self.results.append(newdist)
            t += 1
        self.time = t
        print("finished")
        return self.results

    
    ####################### ANALYSIS ##################################
    
    
    def gini(self,p):

        "Gini coefficient (equation from wikipedia)"

        y = sorted(p)
        n = len(p)

        numer = 2 * sum((i+1) * y[i] for i in range(n))
        denom = n * sum(y)

        return (numer / denom) - (n + 1) / n


    def hist(self, population, label='Dist', **kwargs):
        label = label + ': G=' + str(round(self.gini(population), 2))

        h = plt.hist(list(population), alpha = 0.4, bins = 40, label= label, **kwargs)

        plt.xlabel('Wealth')
        plt.ylabel('Count')
        plt.grid(True)

        plt.legend()


    #NOTE: Eli, please update this
    def show(self, percentiles=(1, 10, 50, 90, 99), **kwargs):

        '''
        print statistics and display a plot and histogram.

        '''

        if self.time == 0:
            print("ERROR: HASN'T RUN YET!")
            return

        # Statistics

        print('   t    Gini stdev' + (' {:3d}%' * len(percentiles)).format(*percentiles))


        fmt = '{:7,d} {:.2f} {:5.1f}' + ' {:4.0f}' * len(percentiles)
  
        initsum = self.N*self.mu
        t = 0
        for pop in self.results:
            t += 1
            pop = sorted(pop)
            if t % (4 * self.N) == 0:
                data = [self.percent(pct, pop) for pct in percentiles]
                print(fmt.format(t, self.gini(pop), statistics.stdev(pop), *data))


    #     #plot individual trajectory
    #     plt.hold(True)
    #     plt.xlabel('Time')
    #     plt.ylabel('Wealth')
    #     plt.grid(True)
    #     plt.plot(times, lists)

    #     plt.show()





        # Plot

    #     plt.xlabel('Time')
    #     plt.ylabel('Wealth')
    #     plt.grid(True)

    #     for pct in percentiles:
    #         #TODO: this should average over all people in different percentiles, or make a separate thing to do that
    #         line = [percent(pct, pop) for pop in results]
    #         plt.plot(times, line)

    #     plt.show()


        # Histogram
        start = self.results[0]
        pop = self.results[-1]
        #R = (min(pop), max(pop))
        R = (min(pop+start), max(pop+start))
        self.hist(start, 'Initial', range=R)
        self.hist(pop, 'Final', range=R)
        plt.show()
        plt.close()
        
        

    def normalize(self,numbers, mu):

        '''
        Positive values, and scale them so they have mean mu.

        '''

        numbers = [max(0, n) for n in numbers]

        factor = len(numbers) * mu / sum(numbers)

        return [x * factor for x in numbers]

    def samples(self,distribution, *args, n = None, mu = None):
        '''
        Sample from the distribution n times,
        then normalize results.

        '''
        if not n:
            n = self.N
        if not mu:
            mu = self.mu

        numbers = [distribution(*args) for _ in range(n)]
        return self.normalize(numbers, mu)


    def percent(self,pct, items):
        '''
        The item that is pct percent through the sorted list of items.

        '''
        return items[min(len(items)-1, len(items) * pct // 100)]



    def animate_distributions(self,times=[0,1],n_frames=100,average=True):
        """
        
        Inputs:
            times. list of ints (length = 2). starting and ending times to run
            n_frames. int. number of frames in animation
            average. bool. If true, will average distributions over all distributions between each timeframe.
                Number of averaged time_steps = round(times[1]/n_frames))
        
        Output: 
            animation object
        
        animates distributions over time in simulation
        
        """

        pop_series = self.results
        run_times = []

        duration = times[1]-times[0]
        tstep = round(float(duration)/n_frames)

        max_t = self.time

        fig = plt.figure()
        def animate(i):
            #label = label + ': G=' + str(round(gini(population), 2))
            plt.cla()
            time = times[0]+i*tstep
            if time < max_t:
                # take average 
                if average:
                    plotvals = np.average(pop_series[time:(time+tstep)],axis=0)
                else:
                    plotvals = pop_series[time]
                plt.hist(plotvals, alpha = 0.4, bins = 30, label=str(time))
                plt.legend()
                plt.xlabel('Wealth')
                plt.ylabel('Count')
                plt.grid(True)

        anim = FuncAnimation(fig, animate, #init_func=init,
                                       frames=n_frames)#), interval=20)#, blit=True)
        HTML(anim.to_jshtml())
        return anim



    def test_stability(self,n_steps=50,end_time=1000):
        
        """
        Inputs:
            n_steps: number of steps over which to analyze stability of distribution
            end_time: last time to analyze.
            
        produces histograms of standard deviations of each agent's wealth over last n_steps steps up until time end_time
        
        """
        
        pop_series = np.array(self.results)
        t_series = pop_series[(end_time-n_steps):end_time]
        stds = []
        mus = []
        for agent_n in range(len(t_series[0])):
            agent_series = t_series[:,agent_n]
            std = np.std(agent_series)
            stds.append(std)
            mus.append(np.mean(agent_series))
        plt.figure()
        plt.hist(stds)
        xlbl = "standard deviations of wealth (unit=absolute wealth); t=["+str(end_time-n_steps)+","+str(end_time)+"]"
        plt.xlabel(xlbl)
        plt.show()


        stds,mus = np.array(stds),np.array(mus)
        plt.figure()
        plt.hist((stds/mus))
        xlbl = "standard deviations of wealth (unit=percent wealth); t=["+str(end_time-n_steps)+","+str(end_time)+"]"
        plt.xlabel(xlbl)
        plt.show()
        
        plt.close()
        
        


In [7]:
# General_Model_MultiCity example:

# meta_sim = General_Model_MultiCity()
# meta_sim.run_many_pops(Pure_Random)
# anim = meta_sim.animate_population_dists()
# HTML(anim.to_jshtml())